# 確率単体上への射影アルゴリズム

参考：
* [Projection onto the probability simplex: An efficient algorithm with a simple proof, and an application](https://arxiv.org/abs/1309.1541)

$D$次元ベクトル$\mathbf{y}=\left[y_1, \ldots, y_D\right]^{\top} \in \mathbb{R}^D$を確率単体上に射影する状況を考えましょう．つまり，次を解きます：

$$
\begin{aligned}
\min _{\mathbf{x} \in \mathbb{R}^D} & \frac{1}{2}\|\mathbf{x}-\mathbf{y}\|^2 \\
\text { s.t. } & \mathbf{x}^{\top} \mathbf{1}=1 \\
& \mathbf{x} \geq \mathbf{0} .
\end{aligned}
$$

これは制約付き二次最適化問題なので，唯一の解を持ちます．その解を$\mathbf{x}=[x_1, \dots, x_D]^\top$としましょう．
これは次のアルゴリズムで効率よく解けます：

---

1. $\mathbf{y}$をソートして，その結果を$\mathbf{u}$とします：$u_1\geq u_2\geq \dots u_D$
2. $\rho=\max \left\{1 \leq j \leq D: u_j+\frac{1}{j}\left(1-\sum_{i=1}^j u_i\right)>0\right\}$
3. $\lambda=\frac{1}{\rho}\left(1-\sum_{i=1}^\rho u_i\right)$
4. $x_i=\max \left\{y_i+\lambda, 0\right\}, i=1, \ldots, D$を返します．

---


In [5]:
import jax
import jax.numpy as jnp

@jax.jit
def projection_to_simplex(y):
    """project y to a probability simplex
    see：https://arxiv.org/pdf/1309.1541
    Args:
        y (jnp.ndarray): (A)-vector

    Returns:
        x (jnp.ndarray): (A)-vector
    """
    D = len(y)
    u = jnp.sort(y)[::-1]
    u_sum = jnp.cumsum(u)
    rho_pos_flag = (u + (1 - u_sum) / (jnp.arange(D) + 1)) > 0
    rho = jnp.argmax(jnp.cumsum(rho_pos_flag))
    lam = (1 - u_sum[rho]) / (rho + 1)
    x = jnp.maximum(y + lam, 0)
    return x


y = jnp.array([-1.0, 2.0, 0.23])
print(projection_to_simplex(y))

y = jnp.array([0.2, 0.95, 0.35])
print(projection_to_simplex(y))


[0. 1. 0.]
[0.03333333 0.7833333  0.18333332]


## アルゴリズムの保証

改めて最適化問題を見てみましょう：

$$
\begin{aligned}
\min _{\mathbf{x} \in \mathbb{R}^D} & \frac{1}{2}\|\mathbf{x}-\mathbf{y}\|^2 \\
\text { s.t. } & \mathbf{x}^{\top} \mathbf{1}=1 \\
& \mathbf{x} \geq \mathbf{0} .
\end{aligned}
$$

この最適化について，次のラグランジュ関数が考えられます：

$$
\mathcal{L}(\mathbf{x}, \lambda, \boldsymbol{\beta})=\frac{1}{2}\|\mathbf{x}-\mathbf{y}\|^2-\lambda\left(\mathbf{x}^{\top} \mathbf{1}-1\right)-\boldsymbol{\beta}^{\top} \mathbf{x}
$$

ここで，
* $\lambda$は和を$1$にする等式制約用
* $\boldsymbol{\beta}=\left[\beta_1, \ldots, \beta_D\right]^{\top}$は$\mathbf{x} \geq 0$用

です．目的と制約が連続かつ１階微分可能なので，[KKT条件](OPT_constraint.ipynb)を思い出すと：
$$
\begin{aligned}
&\nabla f(x^*) + \lambda_1^* \nabla g_1(x^*) = 0\\
&g_1(x^*) \leq 0\\
&\lambda_1^* \geq 0\\
&\lambda_1^* g_1(x^*) = 0\\
\end{aligned}
$$
が成り立たないといけないので，最適な$\mathbf{x}$については
$$
\begin{aligned}
x_i-y_i-\lambda-\beta_i & =0, & & i=1, \ldots, D \\
x_i & \geq 0, & & i=1, \ldots, D \\
\beta_i & \geq 0, & & i=1, \ldots, D \\
x_i \beta_i & =0, & & i=1, \ldots, D \\
\sum_{i=1}^D x_i & =1 . & &
\end{aligned}
$$

が成り立ちます．
よって，
* $x_i > 0$ならば$\beta_i=0$かつ$x_i=y_i + \lambda > 0$が成り立つ
* $x_i = 0$ならば$\beta_i \geq 0$かつ$x_i=y_i+\lambda+\beta_i=0$であり，$y_i+\lambda=-\beta_i \leq 0$が成り立つ．

明らかに，$y_i + \lambda$のうち$\lambda$は定数なので，$x_i=0$は小さい$y_i$に相当します．
ここで，$\textbf{y}$を一般性を失わずに並び替えましょう．
また，同じ並び替えを$\textbf{x}$にも当てはめます：

$$
\begin{array}{r}
y_1 \geq \cdots \geq y_\rho \geq y_{\rho+1} \geq \cdots \geq y_D \\
x_1 \geq \cdots \geq x_\rho>x_{\rho+1}=\cdots=x_D
\end{array}
$$

ここで，$x_1 \geq \cdots \geq x_\rho>0, x_{\rho+1}=\cdots=x_D=0$としました．
つまり，$\rho$は$\mathbf{x} > 0$であるような要素の数です．

最後に，$\mathbf{x}$は和が１にならないといけないので，
$$
1=\sum_{i=1}^D x_i=\sum_{i=1}^\rho x_i=\sum_{i=1}^\rho\left(y_i+\lambda\right)
$$
です．よって，$\lambda=\frac{1}{\rho}\left(1-\sum_{i=1}^\rho y_i\right)$が得られます．

あとは$\rho$を得るだけです．

---

**$\rho$の計算の仕方**

$\rho$を$\mathbf{x}$のうち，正の値を取る解の数とすると，

$$
\rho=\max \left\{1 \leq j \leq D: y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)>0\right\}
$$

が成立します．

**証明**

KKT条件から，
* $i=1, \ldots, \rho$について，$\lambda \rho=1-\sum_{i=1}^\rho y_i$および$y_i+\lambda>0$
* $i=\rho+1, \dots, D$について，$y_i + \lambda < 0$

ですね．これを使って，$j\leq \rho$と$j>\rho$で$y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)$の符号が切り替わることを示します．

**$j=\rho$のとき**

$$
y_\rho+\frac{1}{\rho}\left(1-\sum_{i=1}^\rho y_i\right)=y_\rho+\lambda=x_\rho>0 .
$$
なので，符号は正です．

**$j<\rho$のとき**

$$
\begin{array}{r}
y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+\sum_{i=j+1}^\rho y_i+1-\sum_{i=1}^\rho y_i\right)=\frac{1}{j}\left(j y_j+\sum_{i=j+1}^\rho y_i+\rho \lambda\right) \\
=\frac{1}{j}\left(j\left(y_j+\lambda\right)+\sum_{i=j+1}^\rho\left(y_i+\lambda\right)\right)
\end{array}
$$

$y_i + \lambda > 0$が$i=j\dots\rho$で成立するので，符号は$>0$です．

**$j>\rho$のとき**

$$
\begin{aligned}
y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+1-\sum_{i=1}^\rho y_i-\sum_{i=\rho+1}^j y_i\right) & =\frac{1}{j}\left(j y_j+\rho \lambda-\sum_{i=\rho+1}^j y_i\right) \\
& =\frac{1}{j}\left(\rho\left(y_j+\lambda\right)+\sum_{i=\rho+1}^j\left(y_j-y_i\right)\right) .
\end{aligned}
$$

$y_i +\lambda \leq 0$が$j>\rho$で成り立ち，$\mathbf{y}$がソートされているので，
符号は$<0$です．

---

# 別の射影

個人的に興味があるので，次の射影を考えてみます：

$\mathbf{y}$の他に，さらに$N$個のベクトル$\{\mathbf{v}^n\}_{n \in [1, N]}$を考えます．次の射影を考えましょう：

$$
\begin{aligned}
\min _{\mathbf{x} \in \mathbb{R}^D} & \sum_{n=1}^N |\mathbf{x}^\top \mathbf{v}^n-\mathbf{y}^\top \mathbf{v}^n| + \frac{\alpha}{2} \|\mathbf{x}\|_2\\
\text { s.t. } & \mathbf{x}^{\top} \mathbf{1}=1 \\
& \mathbf{x} \geq \mathbf{0} .
\end{aligned}
$$

ここで$\frac{\alpha\|}{2} \mathbf{x}\|_2$を使って二次計画問題にして解を一意にしています．
これは絶対値が付いた最適化なので，次の形式に直せます：

$$
\begin{aligned}
\min _{\mathbf{t} \in \mathbb{R}^N,\; \mathbf{x} \in \mathbb{R}^D} & \sum_{n=1}^N t_n  + \frac{\alpha}{2} \|\mathbf{x}\|_2\\
\text { s.t. } & 
\mathbf{x}^{\top} \mathbf{1}=1 \\
& \mathbf{x} \geq \mathbf{0} \\
& t_n + \mathbf{x}^\top \mathbf{v}^n \geq c_n \quad \forall n \in [1, N]\\
& t_n - \mathbf{x}^\top \mathbf{v}^n  \geq - c_n
\quad \forall n \in [1, N]
\end{aligned}
$$

ここで，$c_n = \mathbf{y}^\top \mathbf{v}^n$と置きました．
最後の部分は$- t_n \leq \mathbf{x}^\top \mathbf{v}^n - c_n \leq t_n$と同じです．
<!-- これはただの線形計画問題です．最適な解を$\mathbf{z}^*=[t_1,\dots, t_N, x_1, \dots, x_D]^\top$としましょう． -->
<!-- このとき，


$$
\begin{aligned}
\min _{\mathbf{z} \in \mathbb{R}^{N+D}} & \mathbf{z}^\top \mathbf{e}_{1:N} \\
\text { s.t. } & 
\mathbf{x}^{\top} \mathbf{1}=1 \\
& \mathbf{z}_{N+1:D} \geq \mathbf{0} \\
& -t_n +\mathbf{y}^\top \mathbf{v}^n \leq \mathbf{x}^\top \mathbf{v}^n\leq t_n +\mathbf{y}^\top \mathbf{v}^n.
\end{aligned}
$$ -->

## アルゴリズムの導出

変形した線形最適化について，次のラグランジュ関数が考えられます：

$$
\begin{aligned}
\mathcal{L}(\mathbf{z}, \lambda, \boldsymbol{\beta}, \boldsymbol{\gamma}, \boldsymbol{\psi})
=
&\frac{\alpha}{2} \|\mathbf{x}\|_2 + \sum^N_{n=1} t_n\\
&-\lambda\left(\mathbf{x}^{\top} \mathbf{1}-1\right)\\
&-\boldsymbol{\beta}^{\top} \mathbf{x}\\
&-\sum_{n=1}^N \gamma_n (t_n + \mathbf{x}^\top \mathbf{v}^n - c_n)\\
&-\sum_{n=1}^N \psi_n (t_n - \mathbf{x}^\top \mathbf{v}^n + c_n)\\
\end{aligned}
$$

目的と制約が連続かつ１階微分可能なので，[KKT条件](OPT_constraint.ipynb)が成り立たないといけないので，最適な値について，
* 制約の満足：
    * $\sum_{i=1}^D x_i =1$
    * $\mathbf{x} \geq 0$
    * $t_n + \mathbf{x}^\top \mathbf{v}^n \geq \mathbf{y}^\top \mathbf{v}^n$ for all $n = 1\dots N$
    * $t_n - \mathbf{x}^\top \mathbf{v}^n \geq -\mathbf{y}^\top \mathbf{v}^n$ for all $n = 1\dots N$
* ラグランジュ係数は非負：
    * $\boldsymbol{\beta, \gamma, \psi} \geq 0$
* 相補性条件：
    * $x_i \beta_i = 0$ for all $i =1\dots D$
    * $\gamma_n (t_n + \mathbf{x}^\top \mathbf{v}^n - \mathbf{y}^\top \mathbf{v}^n) = 0$ for all $n =1\dots N$
    * $\psi_n (t_n - \mathbf{x}^\top \mathbf{v}^n + \mathbf{y}^\top \mathbf{v}^n) = 0$ for all $n =1\dots N$
* 一次の最適性：
    * $t_n$で微分：$1 - \gamma_n -\psi_n = 0$ for all $n = 1\dots N$
    * $x_i$で微分：$\alpha x_i -\lambda -\beta_i - \sum^N_{n=1}\gamma_n v^n_i + \sum^N_{n=1}\psi_n v^n_i= 0$ for all $i = 1\dots D$

が成り立ちます．



---

1次の最適性から$\psi_n$を消すと

* ラグランジュ係数は非負：
    * $\boldsymbol{\beta} \geq 0$ かつ $1 \geq \boldsymbol{\gamma} \geq 0$
* 相補性条件の部分：
    * $\gamma_n (t_n + \mathbf{x}^\top \mathbf{v}^n - c_n) = 0$ for all $n =1\dots N$
    * $(1-\gamma_n) (t_n - \mathbf{x}^\top \mathbf{v}^n + c_n) = 0$ for all $n =1\dots N$
* 一次の最適性：
    * $x_i$で微分：$x_i = \frac{1}{\alpha} \left(\lambda +\beta_i - \sum^N_{n=1}(1-2\gamma_n) v^n_i\right)$ for all $i = 1\dots D$

---

相補性条件について，
* $1 > \gamma_n>0$ のとき，$t_n = \mathbf{x}^\top \mathbf{v}^n - c_n = - \mathbf{x}^\top \mathbf{v}^n + c_n = 0$
* $\gamma_n=0$のとき，２つ目から$t_n = \mathbf{x}^\top \mathbf{v}^n - c_n > 0$.
* $\gamma_n=1$のとき，１つ目から$t_n = c_n - \mathbf{x}^\top \mathbf{v}^n > 0$.

よって，$t_n = |\mathbf{x}^\top \mathbf{v}^n - c_n|$が成り立つ．
<!-- よって，$\mathbf{x}^\top \mathbf{v}^n \neq c_n$のとき，$\gamma = 0$ or $\gamma = 1$ -->

<!-- これを使うと，
1. $\mathbf{x}^\top \mathbf{v}^n > c_n$のとき：$t_n > 0$ なので，１つ目の相補性から$\gamma = 0$．このとき２つ目から$t_n = \mathbf{x}^\top \mathbf{v}^n - c_n > 0$.
2. $\mathbf{x}^\top \mathbf{v}^n < c_n$のとき：$t_n > 0$ なので，２つ目の相補性から$\gamma = 1$．このとき１つ目から$t_n = c_n - \mathbf{x}^\top \mathbf{v}^n > 0$.
3. $\mathbf{x}^\top \mathbf{v}^n = c_n$のとき：上で述べたように，$t_n = \mathbf{x}^\top \mathbf{v}^n - c_n = - \mathbf{x}^\top \mathbf{v}^n + c_n$
 -->

---

### $\gamma$で場合分け

**$\gamma = 0$のとき**

相補性条件の$x\beta = 0$から，
* $x_i > 0$ならば$\beta_i=0$かつ，一次の最適性から，$x_i = \frac{1}{\alpha} \left(\lambda - \sum^N_{n=1}v^n_i\right)$
* $x_i = 0$ならば$\beta_i \geq 0$かつ，$x_i=\frac{1}{\alpha} \left(\lambda +\beta_i + \sum^N_{n=1} v^n_i\right)=0$であり，$\lambda + \sum^N_{n=1} v^n_i = -\beta_i \leq 0$






明らかに，$y_i + \lambda$のうち$\lambda$は定数なので，$x_i=0$は小さい$y_i$に相当します．
ここで，$\textbf{y}$を一般性を失わずに並び替えましょう．
また，同じ並び替えを$\textbf{x}$にも当てはめます：

$$
\begin{array}{r}
y_1 \geq \cdots \geq y_\rho \geq y_{\rho+1} \geq \cdots \geq y_D \\
x_1 \geq \cdots \geq x_\rho>x_{\rho+1}=\cdots=x_D
\end{array}
$$

ここで，$x_1 \geq \cdots \geq x_\rho>0, x_{\rho+1}=\cdots=x_D=0$としました．
つまり，$\rho$は$\mathbf{x} > 0$であるような要素の数です．

最後に，$\mathbf{x}$は和が１にならないといけないので，
$$
1=\sum_{i=1}^D x_i=\sum_{i=1}^\rho x_i=\sum_{i=1}^\rho\left(y_i+\lambda\right)
$$
です．よって，$\lambda=\frac{1}{\rho}\left(1-\sum_{i=1}^\rho y_i\right)$が得られます．

あとは$\rho$を得るだけです．

---

**$\rho$の計算の仕方**

$\rho$を$\mathbf{x}$のうち，正の値を取る解の数とすると，

$$
\rho=\max \left\{1 \leq j \leq D: y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)>0\right\}
$$

が成立します．

**証明**

KKT条件から，
* $i=1, \ldots, \rho$について，$\lambda \rho=1-\sum_{i=1}^\rho y_i$および$y_i+\lambda>0$
* $i=\rho+1, \dots, D$について，$y_i + \lambda < 0$

ですね．これを使って，$j\leq \rho$と$j>\rho$で$y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)$の符号が切り替わることを示します．

**$j=\rho$のとき**

$$
y_\rho+\frac{1}{\rho}\left(1-\sum_{i=1}^\rho y_i\right)=y_\rho+\lambda=x_\rho>0 .
$$
なので，符号は正です．

**$j<\rho$のとき**

$$
\begin{array}{r}
y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+\sum_{i=j+1}^\rho y_i+1-\sum_{i=1}^\rho y_i\right)=\frac{1}{j}\left(j y_j+\sum_{i=j+1}^\rho y_i+\rho \lambda\right) \\
=\frac{1}{j}\left(j\left(y_j+\lambda\right)+\sum_{i=j+1}^\rho\left(y_i+\lambda\right)\right)
\end{array}
$$

$y_i + \lambda > 0$が$i=j\dots\rho$で成立するので，符号は$>0$です．

**$j>\rho$のとき**

$$
\begin{aligned}
y_j+\frac{1}{j}\left(1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+1-\sum_{i=1}^j y_i\right)=\frac{1}{j}\left(j y_j+1-\sum_{i=1}^\rho y_i-\sum_{i=\rho+1}^j y_i\right) & =\frac{1}{j}\left(j y_j+\rho \lambda-\sum_{i=\rho+1}^j y_i\right) \\
& =\frac{1}{j}\left(\rho\left(y_j+\lambda\right)+\sum_{i=\rho+1}^j\left(y_j-y_i\right)\right) .
\end{aligned}
$$

$y_i +\lambda \leq 0$が$j>\rho$で成り立ち，$\mathbf{y}$がソートされているので，
符号は$<0$です．

-